In [49]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [128]:
real =torch.Tensor(np.load('real.npy')).long().cuda()
real_attn = torch.Tensor(np.load('real_attn.npy')).long().cuda()
fake = torch.Tensor(np.load('fake.npy')).cuda()
fake_attn = torch.Tensor(np.load('fake_attn.npy')).long().cuda()
pred_fake =torch.Tensor( np.load('pred_fake.npy'))
pred_real = torch.Tensor(np.load('pred_real.npy'))

In [129]:
model(real,real_attn)

tensor([[0.8991],
        [0.9323],
        [0.7042]], device='cuda:0', grad_fn=<ReluBackward0>)

In [130]:
model(fake,fake_attn)

tensor([[0.0240],
        [0.0297],
        [0.1063]], device='cuda:0', grad_fn=<ReluBackward0>)

In [137]:
d(torch.argmax(fake,-1))

['Erst schaffthat gerade gerade der ESn demokratischen ParteiFraktion über solchevöllig e',
 'Es sollte nicht dar die foriebung der prioritäten und deriken bei die',
 'Ichens bitte ich die Herrn Kommissar daran,- und hierich']

In [121]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [138]:
en(['.'])
label = en(['Somit hatte die Kommission bereits genügend Zeit, ihr Programm zu erarbeiten, und wir, um es kennenlernen und den Bürgern erklären zu können.',
 'Frau Präsidentin, ich möchte zunächst darauf hinweisen, daß das, was Herr Poettering da sagt, nicht ganz logisch ist.',
 '(Das Parlament lehnt den Antrag ab.) Die Präsidentin.'])
pred = en(['Erst schaffthat gerade gerade der ESn demokratischen ParteiFraktion über solchevöllig e',
 'Es sollte nicht dar die foriebung der prioritäten und deriken bei die',
 'Ichens bitte ich die Herrn Kommissar daran,- und hierich'])

In [147]:
real

tensor([[  264,  1538,  3310,    67,  5837,  5451,  3360,     3, 28078,  1743,
             6,  1650,  7106,   170,     3,    49,  7987,     6,    64,   558,
             6,   561,     3,    15,     7, 31844,    64,   177, 11726,    29,
         24728,   170,   730,     5,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [139]:
label

(tensor([[  264,  1538,  3310,    67,  5837,  5451,  3360,     3, 28078,  1743,
              6,  1650,  7106,   170,     3,    49,  7987,     6,    64,   558,
              6,   561,     3,    15,     7, 31844,    64,   177, 11726,    29,
          24728,   170,   730,     5,     1],
         [ 7672,     3, 20962,    77,     6,     3,   362,  6509,   170, 10151,
           6101,     3, 31306,    35,     6,     3,    26,  7118,   211,     6,
             47,  8816,  1908, 15583,    53,   836, 11865,     6,   311,  1897,
              3, 25407,   229,     5,     1],
         [   41, 17266, 13636,     3, 29723,   177, 20647,   703,     5,    61,
            316,     3, 20962,    77,     5,     1,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1

In [146]:
torch.argmax(fake,-1)

tensor([[ 9344, 23336,   547,  6034,  6034,    74,     3,  3205,    29,     3,
         30677,  1779, 17797, 31002,   510,  6466, 17139,     3,    15],
        [ 1122,  3402,   311,   649,    67,    21,    23,    15, 11848,    74,
          1884,  4130,    35,    64,    74,    23,  2217,   468,    67],
        [ 1674,    35,     7,  9633,     3,   362,    67,  8816,    29,  5837,
         11502,   291,     3, 13702,     6,    18,    64,  1382,   362]],
       device='cuda:0')

In [140]:
pred

(tensor([[ 9344, 23336,   547,  6034,  6034,    74,     3,  3205,    29,     3,
          30677,  1779, 17797, 31002,   510,  6466, 17139,     3,    15,     1],
         [ 1122,  3402,   311,   649,    67,    21,    23,    15, 11848,    74,
           1884,  4130,    35,    64,    74,    23,  2217,   468,    67,     1],
         [   27,  1559,     7,  9633,     3,   362,    67,  8816,    29,  5837,
          11502,   291,     3, 13702,     6,    18,    64,  1382,   362,     1]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [141]:
model = torch.load('./model/D_A.pt').train()

In [142]:
#real sentences
x = label[0].cuda()
#delte 0 give huge diff
       
# x = torch.Tensor(x).long().cuda()
print(x)
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_label =  model.classifier(distr)#(bs,1)
print(ret_label)

print(model(x,x_attn))

tensor([[  264,  1538,  3310,    67,  5837,  5451,  3360,     3, 28078,  1743,
             6,  1650,  7106,   170,     3,    49,  7987,     6,    64,   558,
             6,   561,     3,    15,     7, 31844,    64,   177, 11726,    29,
         24728,   170,   730,     5,     1],
        [ 7672,     3, 20962,    77,     6,     3,   362,  6509,   170, 10151,
          6101,     3, 31306,    35,     6,     3,    26,  7118,   211,     6,
            47,  8816,  1908, 15583,    53,   836, 11865,     6,   311,  1897,
             3, 25407,   229,     5,     1],
        [   41, 17266, 13636,     3, 29723,   177, 20647,   703,     5,    61,
           316,     3, 20962,    77,     5,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]], device='cuda:0')
torch.Size([3, 35])
		---------embedding
torch.Size([3, 35, 512])
tensor([[[ 18.1126,   3.8309,  28.8626,  ...,   1.6434, -15.6748,  17.

In [143]:
#real sentences
x = pred[0].cuda()
       
# x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
print(torch.sum(distr,1).shape)
print(torch.sum(x_attn,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_pred =  model.classifier(distr)#(bs,1)
print(ret_pred)

torch.Size([3, 20])
		---------embedding
torch.Size([3, 20, 512])
tensor([[[ -4.5809, -28.1126,  21.9876,  ...,  -5.9559, -39.7358,   4.1746],
         [ 18.3626, -15.1748,  -6.3934,  ...,   2.0028,  23.9876,  14.9873],
         [ 15.1748,  -6.5184, -12.6748,  ...,   3.5496,  -7.9246,  -6.7684],
         ...,
         [ 11.2373,   8.0497,  14.1748,  ...,   9.7998,  -7.8621,  -3.5965],
         [ -7.9871,   6.1121,  18.4876,  ...,  14.2373,  14.7373,  -2.3465],
         [ 12.6122,   8.1747, -11.6122,  ...,   7.9247,  -7.2996,   0.9325]],

        [[ -2.1903,  -1.7606,  12.6123,  ..., -29.1126, -12.4873,  21.3626],
         [ 17.4876, -18.8626,   1.7528,  ...,  11.6748,  11.3623,  19.4876],
         [ -7.7996,   2.9246,  12.8623,  ...,  -9.0498, -31.4876,   9.1748],
         ...,
         [ -1.2293,  -4.7996,  -1.9871,  ...,   9.6748,  34.9858,   7.5496],
         [  8.4248,   2.3153,  22.3626,  ...,  14.8623,  -4.7371,  -1.4246],
         [ 12.6122,   8.1747, -11.6122,  ...,   7.9247,  

In [144]:
criterionGAN = torch.nn.MSELoss()
pred_real = ret_label
loss_D_real = criterionGAN(pred_real, torch.ones((pred_real.shape[0],1),device='cuda'))
print(loss_D_real)
# Fake
pred_fake = ret_pred
loss_D_fake = criterionGAN(pred_fake, torch.zeros((pred_fake.shape[0],1),device='cuda'))
# Combined loss and calculate gradients
print(loss_D_fake)


loss_D = (loss_D_real + loss_D_fake) * 0.5
print(loss_D)

tensor(0.0427, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.6226, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3327, device='cuda:0', grad_fn=<MulBackward0>)


,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier

## FAKE dont end with 1, cuz i use model.generate but not self.generate